In [292]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [293]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers

In [294]:
# Loading data
dataset  = pd.read_csv("bank.csv")

In [295]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [296]:
#Checking unique values to decide categorical attributes vs others
dataset.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [297]:
# Checking null values and datatypes
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [298]:
# Drop RowNumber and CustomerId because it won't be useful in predictive task
dataset.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [299]:
# Convert object dtype into category
cat_features = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember', 'NumOfProducts', 'Tenure']
for colname in cat_features:
    dataset[colname] = dataset[colname].astype('category')

In [300]:
#
dataset.columns
num_features = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']

In [301]:
#Normalizing the Non categorical features
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
standardized_df = pd.DataFrame(ss.fit_transform(dataset[num_features]), columns = num_features)
standardized_df['Exited'] = dataset['Exited']
standardized_df.head()

,CreditScore,Age,Balance,EstimatedSalary,Exited
0,-0.326221,0.293517,-1.225848,0.021886,1
1,-0.440036,0.198164,0.117350,0.216534,0
2,-1.536794,0.293517,1.333053,0.240687,1
3,0.501521,0.007457,-1.225848,-0.108918,0
4,2.063884,0.388871,0.785728,-0.365276,0


In [302]:
# Encoding for categorical data
from sklearn.preprocessing import LabelEncoder
le_geography = LabelEncoder()
le_gender = LabelEncoder()
le_HasCrCard = LabelEncoder()
le_IsActiveMember = LabelEncoder()
le_NumOfProducts = LabelEncoder()
le_Tenure = LabelEncoder()

le_df = pd.DataFrame()

le_df['Geography'] = le_geography.fit_transform(dataset['Geography'])
le_df['Gender'] = le_gender.fit_transform(dataset['Gender'])
le_df['HasCrCard'] = le_HasCrCard.fit_transform(dataset['HasCrCard'])
le_df['IsActiveMember'] = le_IsActiveMember.fit_transform(dataset['IsActiveMember'])
le_df['NumOfProducts'] = le_NumOfProducts.fit_transform(dataset['NumOfProducts'])
le_df['Tenure'] = le_Tenure.fit_transform(dataset['Tenure'])

le_df.head()

,Geography,Gender,HasCrCard,IsActiveMember,NumOfProducts,Tenure
0,0,0,1,1,0,2
1,2,0,0,1,0,1
2,0,0,1,0,2,8
3,0,0,0,0,1,1
4,2,0,1,1,0,2


In [303]:
# Appending scalled attributes DF with encoded columns
model_df = pd.concat([standardized_df,le_df],axis=1)
model_df.head()

,CreditScore,Age,Balance,EstimatedSalary,Exited,Geography,Gender,HasCrCard,IsActiveMember,NumOfProducts,Tenure
0,-0.326221,0.293517,-1.225848,0.021886,1,0,0,1,1,0,2
1,-0.440036,0.198164,0.117350,0.216534,0,2,0,0,1,0,1
2,-1.536794,0.293517,1.333053,0.240687,1,0,0,1,0,2,8
3,0.501521,0.007457,-1.225848,-0.108918,0,0,0,0,0,1,1
4,2.063884,0.388871,0.785728,-0.365276,0,2,0,1,1,0,2


In [305]:
# Exited column is our target attribute and rest are featurs
X = model_df.drop(['Exited'],axis=1)
y = model_df['Exited']

In [306]:
model_df.head()

,CreditScore,Age,Balance,EstimatedSalary,Exited,Geography,Gender,HasCrCard,IsActiveMember,NumOfProducts,Tenure
0,-0.326221,0.293517,-1.225848,0.021886,1,0,0,1,1,0,2
1,-0.440036,0.198164,0.117350,0.216534,0,2,0,0,1,0,1
2,-1.536794,0.293517,1.333053,0.240687,1,0,0,1,0,2,8
3,0.501521,0.007457,-1.225848,-0.108918,0,0,0,0,0,1,1
4,2.063884,0.388871,0.785728,-0.365276,0,2,0,1,1,0,2


In [307]:
# Splitting the dataset into the Training and Testing set.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 7)

In [308]:
print(f'training shapes: {X_train.shape}, {y_train.shape}')
print(f'testing shapes: {X_test.shape}, {y_test.shape}')

training shapes: (8000, 10), (8000,)
testing shapes: (2000, 10), (2000,)


In [333]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

dt_clf = DecisionTreeClassifier(max_depth=5,max_features='sqrt')

dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

           0       0.86      0.97      0.91      1589
           1       0.78      0.37      0.50       411

    accuracy                           0.85      2000
   macro avg       0.82      0.67      0.70      2000
weighted avg       0.84      0.85      0.83      2000



In [331]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
pred = neigh.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.85      0.92      0.88      1589
           1       0.55      0.38      0.45       411

    accuracy                           0.81      2000
   macro avg       0.70      0.65      0.67      2000
weighted avg       0.79      0.81      0.80      2000



In [309]:
model1 = Sequential()

from tensorflow.keras.metrics import Recall

In [310]:
model1.add(Dense(32, input_shape = (10,), activation = 'relu'))
model1.add(Dense(16, activation = 'relu'))
model1.add(Dense(16, activation = 'relu'))
model1.add(Dense(1, activation = 'sigmoid'))

In [311]:
sgd = optimizers.Adam(lr = 0.001)

In [312]:
model.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics=["accuracy"])

In [313]:
model1.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 32)                352       
_________________________________________________________________
dense_61 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_62 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 17        
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [314]:
model1.fit(X_train, y_train.values, batch_size = 20, epochs = 50, verbose = 1)

Train on 8000 samples
Epoch 1/50
8000/8000 [==============================] - 2s 220us/sample - loss: 0.4905 - accuracy: 0.7790
Epoch 2/50
8000/8000 [==============================] - 1s 181us/sample - loss: 0.4236 - accuracy: 0.8080
Epoch 3/50
8000/8000 [==============================] - 1s 173us/sample - loss: 0.4004 - accuracy: 0.8244
Epoch 4/50
8000/8000 [==============================] - 1s 174us/sample - loss: 0.3736 - accuracy: 0.8396
Epoch 5/50
8000/8000 [==============================] - 2s 189us/sample - loss: 0.3584 - accuracy: 0.8509
Epoch 6/50
8000/8000 [==============================] - 1s 186us/sample - loss: 0.3525 - accuracy: 0.8529
Epoch 7/50
8000/8000 [==============================] - 1s 176us/sample - loss: 0.3460 - accuracy: 0.8559
Epoch 8/50
8000/8000 [==============================] - 1s 173us/sample - loss: 0.3431 - accuracy: 0.8577
Epoch 9/50
8000/8000 [==============================] - 1s 174us/sample - loss: 0.3422 - accuracy: 0.8566
Epoch 10/50
8000/8000 [=

In [315]:
#predicting the results of model

y_pred = model1.predict(X_test)
y_pred = (y_pred > 0.5) #to classify each probability into True or False

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print (cm, '\n\n', y_pred[:5, :])

[[1500   89]
 [ 210  201]] 

 [[False]
 [False]
 [False]
 [ True]
 [False]]


In [325]:
#accuracy of model
print ((1500 + 201)/2000)

0.8505


In [317]:
#Model looks good and slightly overfit, Now lets adjust hyperparameter and see if can get better result for moodel2

In [318]:
model2 = Sequential()
model2.add(Dense(32, input_shape = (10,), activation = 'relu'))
model2.add(Dense(16, activation = 'relu'))
model2.add(Dense(16, activation = 'relu'))
model2.add(Dense(1, activation = 'sigmoid'))

In [319]:
sgd = optimizers.Adam(lr = 0.001)

In [320]:
model2.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics=["accuracy"])

In [321]:
model2.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 32)                352       
_________________________________________________________________
dense_65 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_66 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_67 (Dense)             (None, 1)                 17        
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [322]:
model2.fit(X_train, y_train.values, batch_size = 200, epochs = 50, verbose = 2)

Train on 8000 samples
Epoch 1/50
8000/8000 - 1s - loss: 0.5224 - accuracy: 0.7968
Epoch 2/50
8000/8000 - 0s - loss: 0.4600 - accuracy: 0.7971
Epoch 3/50
8000/8000 - 0s - loss: 0.4390 - accuracy: 0.8025
Epoch 4/50
8000/8000 - 0s - loss: 0.4310 - accuracy: 0.8066
Epoch 5/50
8000/8000 - 0s - loss: 0.4272 - accuracy: 0.8133
Epoch 6/50
8000/8000 - 0s - loss: 0.4225 - accuracy: 0.8151
Epoch 7/50
8000/8000 - 0s - loss: 0.4184 - accuracy: 0.8200
Epoch 8/50
8000/8000 - 0s - loss: 0.4142 - accuracy: 0.8259
Epoch 9/50
8000/8000 - 0s - loss: 0.4084 - accuracy: 0.8296
Epoch 10/50
8000/8000 - 0s - loss: 0.4024 - accuracy: 0.8303
Epoch 11/50
8000/8000 - 0s - loss: 0.3960 - accuracy: 0.8316
Epoch 12/50
8000/8000 - 0s - loss: 0.3894 - accuracy: 0.8340
Epoch 13/50
8000/8000 - 0s - loss: 0.3847 - accuracy: 0.8374
Epoch 14/50
8000/8000 - 0s - loss: 0.3804 - accuracy: 0.8381
Epoch 15/50
8000/8000 - 0s - loss: 0.3764 - accuracy: 0.8407
Epoch 16/50
8000/8000 - 0s - loss: 0.3743 - accuracy: 0.8415
Epoch 17/50

In [323]:
#predicting the results using model1

y_pred = model2.predict(X_test)
y_pred = (y_pred > 0.5) #to classify each probability into True or False

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print (cm, '\n\n', y_pred[:5, :])

[[1475  114]
 [ 187  224]] 

 [[ True]
 [False]
 [False]
 [ True]
 [False]]


In [326]:
#accuracy of model2
print ((1475 + 224)/2000)

0.8495


In [73]:
#We found this model2 better slightly than model1